In [3]:
import numpy as np
from scipy.integrate import odeint

In [4]:
# Define Variables and parameters
def lupus_periodic_flare(init = (0.1, 0.4, 1.7, 0.1), # tuple: (I0, P0, D0, A0)
                         tmax=600, # number of days to simulate
                         sep = 45, # num of days in between flares
                         flare_length = 10, # num of days a flare lasts
                         si = 0.002, # base rate that immune complexes deposit in the kidneys
                         sid = 0.015, # base immune response to accumulation of damaged cells
                         kid = 1,
                         kip = 0.025, #rate of immune complex removal from system
                         kpi = 0.13, #rate of mediator activation and recruitment
                         kpp = 0.02, #immune response amplified by existing inflammatory response (kpi)
                         kpd = 0.001, #rate of activation for pro-inflammatory agents as a result of cytokine release or induced by damaged tissue
                         mup = 0.06, #decay of pro-inflammatory mediators
                         kdip = 0.025, #rate of phagocytosis of immune complexes by immune cells
                         kdp = 0.27, #rate at which collateral damage is produced by pro-inflammatory mediators 
                         mud = 0.04, #decay rate of damage
                         sa = 0.05, #addition of anti-inflammatory drugs
                         kap = 0.022, #intrarenal production of anti-inflammatory mediators
                         kad = 0.22, #intrarenal rate of tissue damage ??
                         mua = 2.2, #rate of anti-inflammatory agent degradation
                         Ainf = 1
                        ):

    t_range = srange(0, tmax, 1)

    def fsid_t(t): #time dependency Sid
        sid = 0.002
        if numpy.mod(t,sep) <= flare_length : #gives the higher value for ~two weeks of time based on data
            sid = sid*5
        else:
            sid = 0.002
        return sid 

    def fsi_t(t): #time dependency Si
        si = 0.001
        if numpy.mod(t,sep) <= flare_length : 
            si = si*5
        else:
            si = 0.001
        return si
    
    def fsa_t(t):
        saf = 3*sa
        return saf if t >=changeday and t<=changeday + flare_length else sa

    def systems (IPDA, t, sa_t, si_t,sid_t, kip_t, kpp_t, kpi_t, kpd_t, kdip_t,kdp_t, kap_t,kad_t,mup_t,mua_t,mud_t,kid_t,Ainf_t):
        I, P, D, A = IPDA 
        
        def f(x):
            return x/((1+A/Ainf)^2) 

        Idot = f(si_t(t)) + f(sid_t(t))*(D^2/(kid_t^2 + D^2)) - kip_t*f(P)*I
        Pdot = f(kpi_t*I + kpp_t*P) + f(kpd_t*D) - mup_t*P
        Ddot = kdip_t*f(P)*I + kdp_t*f(P) - mud_t*D
        Adot = fsa_t(t) + f(kap_t*P + kad_t*D) - mua_t*A

        LupusSystem = (Idot, Pdot, Ddot, Adot)
        return LupusSystem

    IPDAsim = odeint(systems, init, t_range, args=(fsa_t, fsi_t, fsid_t, kip, kpp, kpi, kpd, kdip,kdp, kap,kad,mup,mua,mud,kid, Ainf))
    IPDAsim = np.insert(IPDAsim, 0, t_range, axis=1)

    return IPDAsim

def plot_ts4(sim, title = ""):
    Its = list_plot(sim[::,(0,1)], plotjoined=True, color="grey", legend_label="I")
    Pts = list_plot(sim[::,(0,2)], plotjoined=True, color="blue", legend_label="P")
    Dts = list_plot(sim[::,(0,3)], plotjoined=True, color="red", legend_label="D")
    Ats = list_plot(sim[::,(0,4)], plotjoined=True, color="purple", legend_label="A", title = title)
    return Its+Pts+Dts+Ats

plot_ts4(lupus_periodic_flare(), title = "Periodic Flare")

NameError: name 'srange' is not defined